# Floating Point Accuracy

Even in a modern language like Python 3.12, there is a limited number of bits to store floating point numbers. We can find that limit pretty quickly.

In [1]:
print(3e7 + 3e-7)
print(3e10 + 3e-7)
print(3e10 + 3e-5)

30000000.0000003
30000000000.0
30000000000.00003


Some of the classic floating point math works surprisingly well in Python; someone must have done some wizardry in the back ground. It's a very Pythonic thing to do really.

In [2]:
a = 1/3 + 1/3 + 1/3

print(a)
print(f"{a:5.3f}")
print(int(a))
print(a == 1.0)

1.0
1.000
1
True



But we can still very quickly see where some issues arise.

In [3]:
a = 1/3 + 1/3

print(a)
print(f"{a:20.19f}")
print(int(a))
print(round(a))
print(a == 2/3)
print(a == 0.6666666)
print(a - 0.666666 < 1e5)

print(0.1 + 0.1 +0.1 == 0.3)

0.6666666666666666
0.6666666666666666297
0
1
True
False
True
False


## Example 4.1

Given $x = 1, y = 1 + 10^{-14} \sqrt{2}$, we can see trivially that $10^{14}(y-x) = \sqrt{2}$

The book tells us this calculation isn't very accurate, let's see.

In [4]:
import numpy as np

x = 1.0
y = 1.0 + 1e-14 * np.sqrt(2)
print(1e14 * (y - x))
print(np.sqrt(2))

1.4210854715202004
1.4142135623730951


## Exercise 4.2

You likely completed exercise 4.2 in your previous course, but is worth revisiting. Recall there were two formulae given to calculate roots of a quadratic equation. The classic equation

$$ x = {-b \pm \sqrt{b^2-4ac}\over2a} $$

And an alternative:

$$ x = {2c\over-b\mp\sqrt{b^2-4ac}} $$

Let's explore this quickly. We'll exploit the fact that $\sqrt{b^2-4ac}$ appears in both equations.

In [1]:
def root(b, a, c):
    return (b*b - 4 * a * c)**0.5

def classic(a, b, c):
    d = root(b, a, c)
    return (-b + d) / (2 * a), (-b - d) / (2 * a)

def modified(a, b, c):
    d = root(b, a, c)
    return 2 * c / (-b - d), 2 * c / (-b + d)

def test(roots, solutions):
    a, b, c = roots
    x1, x2 = solutions
    print(a*(x1**2) + b*x1 + c)
    print(a*(x2**2) + b*x2 + c)

# easy
a = 1
b = -8
c = 5

roots = (a, b, c)

print(classic(*roots), modified(a, b, c))

test(roots, classic(*roots))
test(roots, modified(*roots))

(7.3166247903554, 0.6833752096446002) (7.3166247903554, 0.6833752096446002)
0.0
0.0
0.0
0.0


In [2]:
a = 0.001
b = 1000
c = 0.001

roots = (a, b, c)

print(classic(*roots), modified(a, b, c))

test(roots, classic(*roots))
test(roots, modified(*roots))

(-9.999894245993346e-07, -999999.999999) (-1.000000000001e-06, -1000010.5755125057)
1.0575401665491313e-08
7.247924804689582e-08
0.0
10575.62534720993


In [3]:
a = 0.001
b = 1000
c = 0.001
print(root(b,a,c))

999.999999998


At this point hopefully the issue is obvious: the term $\sqrt{b^2-4ac}$ is just going to $b$, so the $-b + ...$ terms are going to 0.

Looking at the form of our equations, the second result from our classic equation should work, and the first from our modified equation, we can test those values individually

In [4]:
test(roots, (-1e-6, -1e6))

1.0000680839006293e-15
0.001


In conclusion: floating point numbers make life difficult. Use with caution